In [2]:
import requests
import regex as re
import json
import time
import random
import networkx as nx

In [3]:
def open_file(filepath:str='cache.json')->dict:
    '''Opens a file'''
    cache_file = open(filepath, 'r')
    cache_contents = cache_file.read()
    artist_data = json.loads(cache_contents)
    cache_file.close()
    return artist_data

In [36]:
def load_full_network(playlist_link:str,random_link:str,cache_file:str)->nx.Graph:
    playlist_network = open_file(playlist_link)
    random_network = open_file(random_link)
    cache = open_file(cache_file)
    # combine!!

    # playlist network doesn't have name key, my bad
    for artist in playlist_network.keys():
        playlist_network[artist]['name']=artist

    g = nx.Graph()
    for artist in playlist_network.keys():
        g.add_node(artist)
        colab = playlist_network[artist]['collaborators'].keys()
        for y in colab:
            if artist !=y:
                if y not in g.nodes:
                    g.add_node(y)
                g.add_edge(artist,y)

    for n in g.nodes():
        g.nodes[n]['source'] = 'playlist'
        if n in playlist_network.keys():
            for key in playlist_network[n].keys():
                g.nodes[n][key]= playlist_network[n][key]
        else:
            for key in cache[n].keys():
                g.nodes[n][key]=cache[n][key]
            g.nodes[n]['in_playlist']=False
            g.nodes[n]['name']=n


    for artist in random_network.keys():
        if artist not in g.nodes:
            g.add_node(artist)
            g.nodes[artist]['source']='random'
        colab = random_network[artist]['collaborators'].keys()
        for y in colab:
            if artist !=y:
                if y not in g.nodes:
                    g.add_node(y)
                g.add_edge(artist,y)
                g.nodes[y]['source']='random'

    for n in g.nodes():
        if n in random_network.keys():
            for key in random_network[n].keys():
                g.nodes[n][key]= random_network[n][key]
            g.nodes[n]['in_playlist']=False
            g.nodes[n]['name']=n
            if n not in playlist_network.keys():
                for key in cache[n].keys():
                    g.nodes[n][key]=cache[n][key]
            g.nodes[n]['source']='random'
    return g

In [37]:
# loading complete networks
week1_g = load_full_network('10_26_playlist_layer2.json','10_26_random_2layer.json','cache.json')
week2_g = load_full_network('11_01_playlist_layer2.json','10_26_random_2layer.json','cache.json')
week3_g = load_full_network('11_08_playlist_layer2.json','10_26_random_2layer.json','cache.json')

Now I want to gather the info about the different networks and see how they change.

In [12]:
# make dataframe with networks attributes
week1_df =pd.DataFrame.from_dict(dict(week1_g.nodes(data=True)), orient='index')
week2_df =pd.DataFrame.from_dict(dict(week2_g.nodes(data=True)), orient='index')
week3_df =pd.DataFrame.from_dict(dict(week3_g.nodes(data=True)), orient='index')
# print the columns
print(week1_df.columns)

Index(['source', 'genres', 'img_info', 'popularity', 'followers',
       'collaborators', 'in_playlist', 'link', 'name', 'api_link'],
      dtype='object')


In [41]:
# making an easy way to iterate
graphs = {'week1': {'graph':week1_g, 'df':week1_df},
          'week2': {'graph':week2_g, 'df':week2_df},
          'week3': {'graph':week3_g, 'df':week3_df},}

In [77]:
for week in graphs.keys():
    # set graph & df
    g, df = graphs[week]['graph'], graphs[week]['df']
    # get pagerank
    pr = nx.pagerank(g)
    df['pagerank']=df['name'].apply(lambda x: pr[x])
    # get closeness centrality
    cc = nx.closeness_centrality(g)
    df['closeness_cent']=df['name'].apply(lambda x: cc[x])
    # clustering
    clust = nx.clustering(g)
    df['clustering']=df['name'].apply(lambda x: clust[x])
    # degree centrality
    deg_cent = nx.degree_centrality(g)
    df['deg_cent']=df['name'].apply(lambda x: deg_cent[x])
    # betweenness centrality
    # btwn_centr = nx.betweenness_centrality(g)
    # df['btwn_centr']=df['name'].apply(lambda x: btwn_centr[x])
    # degree
    deg = nx.degree(g)
    df['degree']=df['name'].apply(lambda x: deg[x])

KeyboardInterrupt: 

In [40]:
playlist_nodes = [x for x in week1_g.nodes if week1_g.nodes[x]['source']=='playlist'] #if week1_g.nodes[x]['source']=='playlist'
random_nodes = [x for x in week1_g.nodes if week1_g.nodes[x]['source']=='random']

In [ ]:
# set weeks in col
graphs['week1']['df']['week']=1
graphs['week2']['df']['week']=2
graphs['week3']['df']['week']=3
# combine
mega = pd.concat([graphs['week1']['df'],graphs['week2']['df'],graphs['week3']['df']],axis=0)

In [81]:
graphs['week3']['df'].head().to_csv('week3.csv')

In [ ]:
mega.groupby(['in_playlist','source','week'])[['pagerank','closeness_cent','clustering','deg_cent','degree']].mean()

pagerank  closeness_cent  clustering  deg_cent  \
in_playlist source   week                                                   
False       playlist 1     0.000027        0.178964    0.065266  0.000092   
                     2     0.000028             NaN    0.064974  0.000093   
                     3     0.000028             NaN    0.065368  0.000093   
            random   1     0.000191        0.212443    0.122278  0.000726   
                     2     0.000191             NaN    0.122448  0.000728   
                     3     0.000191             NaN    0.122448  0.000729   
True        playlist 1     0.000116        0.223681    0.138634  0.000469   
                     2     0.000114             NaN    0.140451  0.000466   
                     3     0.000113             NaN    0.143961  0.000461   

                              degree  
in_playlist source   week             
False       playlist 1      3.333838  
                     2      3.356332  
                     3      3.365170  
            random   1     26.255879  
                     2     26.240471  
                     3     26.240471  
True        playlist 1     16.964789  
                     2     16.798561  
                     3     16.600000

In [63]:
# save to csv for my peers
# graphs['week1']['df'].to_csv('week1.csv')
# graphs['week2']['df'].to_csv('week2.csv')
# graphs['week3']['df'].to_csv('week3.csv')